In [1]:
%pwd

'd:\\Opencv_projects\\chest_classification\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\Opencv_projects\\chest_classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class BaseModelConfig:
    root_dir:Path
    base_model_path:Path
    base_updated_model_path:Path
    params_image_size:list
    params_learning_rate:float
    params_include_top:bool
    params_weight:str
    params_classes:int

In [5]:
from src.CNNChestClassifier.constants import *
from src.CNNChestClassifier.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_base_model_config(self)->BaseModelConfig:
        
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        base_model_config = BaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            base_updated_model_path=Path(config.updated_base_model_path),
            params_image_size= self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weight=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return base_model_config

In [7]:
import os
import urllib.request as requests
from zipfile import ZipFile
import tensorflow as tf
import keras

In [8]:
class PrepareBaseModel:
    def __init__(self,config:BaseModelConfig):
        self.config = config
        
    @staticmethod
    def save_model(path:Path,model:keras.Model):
        model.save(path)

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(include_top=self.config.params_include_top,
        weights=self.config.params_weight,
        input_shape=self.config.params_image_size
        )
        self.save_model(path = self.config.base_model_path,model=self.model)
    @staticmethod
    def _prepare_full_model(base_model,classes,learning_rate):
        base_model.trainable = False
        model = keras.Sequential()
        model.add(base_model)
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(units=classes,activation='softmax'))
        
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=['accuracy'],
        loss=tf.losses.CategoricalCrossentropy())

        model.summary()

        return model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(base_model=self.model,
        classes=self.config.params_classes,
        learning_rate = self.config.params_learning_rate
        )

        self.save_model(path=self.config.base_updated_model_path,model=self.full_model)

In [9]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e
    

[2024-07-15 02:41:24,438: INFO: common: yaml file: config\config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1ooEK-e37Ig48zPBvi27d2AY9lhN34EPW/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}}
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1ooEK-e37Ig48zPBvi27d2AY9lhN34EPW/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_mo

c:\Users\Aryan Kumar\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 2)                 50178     
                                                                 
Total params: 14764866 (56.32 MB)
Trainable params: 50178 (196.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________
